In [58]:
import sqlalchemy
from sqlalchemy import (create_engine, Column, String, Integer, Float, MetaData, 
                        Table, type_coerce, ForeignKey, case)
from sqlalchemy.orm import mapper, create_session, relationship, aliased, Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
from sqlalchemy.sql import and_, or_, not_, func
from sqlalchemy.sql import select
import os
from os.path import isfile
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import datetime
import glob
import gsw
import pickle
import matplotlib as mpl

import sys
sys.path.append('../')
import PARfuns as pf

import scipy.optimize as scopt
from sqlalchemy.types import DateTime

%matplotlib inline

In [2]:
basedir='/data/eolson/results/MEOPAR/PARcalcs/'
dbname='PARCalcs.sqlite'
engine = create_engine('sqlite:///' + basedir + dbname, echo = False)
connection=engine.connect()
# rename tables ( do 1x)
#connection.execute('ALTER TABLE StationTBL RENAME TO DFO_StationTBL;')
#connection.execute('ALTER TABLE ObsTBL RENAME TO DFO_ObsTBL;')
#connection.execute('ALTER TABLE CalcsTBL RENAME TO DFO_CalcsTBL;')

In [3]:
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
# existing tables:
StationTBL=Base.classes.DFO_StationTBL
ObsTBL=Base.classes.DFO_ObsTBL
CalcsTBL=Base.classes.DFO_CalcsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)


In [4]:
PAR=case([(ObsTBL.PAR!=None, ObsTBL.PAR)], else_=
             case([(ObsTBL.PAR1!=None, ObsTBL.PAR1)], else_=ObsTBL.PAR_1))

In [ ]:
SA=case([(CalcsTBL.Salinity_T0_C0_SA!=None, CalcsTBL.Salinity_T0_C0_SA)], else_=
         case([(CalcsTBL.Salinity_T1_C1_SA!=None, CalcsTBL.Salinity_T1_C1_SA)], else_=
         case([(CalcsTBL.Salinity_SA!=None, CalcsTBL.Salinity_SA)], else_= None)))
CT=case([(CalcsTBL.Temperature_Primary_CT!=None, CalcsTBL.Temperature_Primary_CT)], else_=
         case([(CalcsTBL.Temperature_Secondary_CT!=None, CalcsTBL.Temperature_Secondary_CT)], else_=CalcsTBL.Temperature_CT))
ZD=case([(ObsTBL.Depth!=None,ObsTBL.Depth)], else_= CalcsTBL.Z)

In [54]:
Fl=case([(ObsTBL.Fluorescence_URU_Seapoint!=None,ObsTBL.Fluorescence_URU_Seapoint)], 
        else_= ObsTBL.Fluorescence_URU_Wetlabs)

In [ ]:
# 1) get list of stations with PAR data; limit to PAR>=1 because data only saved to 1 decimal place
# 2) for each station, return PAR data sorted by depth
# 3) iterate through 2, saving previous values and calculate k and mean quantities; enter results in new table
#     # beam attenuation from Transmissivity? Cannot verify wavelength but path length always 0.25 or 0.00. Assume really 0.25 always

#things to accumulate: min PAR at level, max PAR at Station

In [47]:
#1)
qst0=session.query(StationTBL.ID.label('StationTBLID'),ZD.label('Z'),ObsTBL.PAR).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                            StationTBL.Lat<51,
                                            StationTBL.Include==True,ObsTBL.Include==True,
                                            ObsTBL.PAR>=1)).group_by(StationTBL.ID,ZD).subquery()

In [48]:
qst1=session.query(qst0.c.StationTBLID,qst0.c.Z,func.count(qst0.c.PAR).label('PARCount')).\
                group_by(qst0.c.StationTBLID,qst0.c.Z).subquery()
qst2=session.query(qst1.c.StationTBLID).filter(qst1.c.PARCount>1)
if len(qst2.all())>0:
    print('There are multiple PAR measurements per depth at these stations:')
    print(qst2.all())

In [49]:
#generate list of stations to process:
stalist=[el[0] for el in session.query(qst0.c.StationTBLID).distinct()]

In [55]:
ista=stalist[0]
profile=pd.DataFrame(session.query(ObsTBL.ID.label('OID'),ZD.label('Z'),ObsTBL.PAR,SA.label('SA'),
                                   CT.label('CT'),Fl.label('Fluor'),ObsTBL.Transmissivity.label('Xmiss')).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(StationTBL.ID==ista,ZD>=0,ObsTBL.PAR>1)).\
                                                           order_by(ZD).all())

In [56]:
profile

,OID,Z,PAR,SA,CT,Fluor,Xmiss
0,28925,1.983092,43.9,28.830853,8.556241,24.087,9.8
1,28926,2.974630,18.2,29.377437,8.268308,22.373,9.6
2,28927,3.966164,7.2,29.903118,7.810667,9.840,12.0
3,28928,4.957693,5.4,29.992099,7.640720,6.347,20.1
4,28929,5.949217,3.8,30.017072,7.589757,5.531,48.3
5,28930,6.940737,2.9,30.024359,7.570646,2.621,58.9
6,28931,7.932251,2.3,30.035465,7.562550,2.580,61.6
7,28932,8.923761,1.9,30.041748,7.564384,1.887,62.3
8,28933,9.915266,1.6,30.048332,7.559674,0.986,62.3
9,28934,10.906766,1.4,30.052039,7.558123,0.964,62.0


In [60]:
maxPAR=np.max(profile['PAR'])

In [ ]:
for i,r in profile.iterrows():
    ic=pf.cXfromX(r['Xmiss'])
    if i>0:
        # some calcs:
        mZ=.5*(r.Z+pZ)
        mSA=.5*(r.SA+pSA)
        mCT=.5*(r.CT+pCT)
        mc=.5*(pc+ic)
        dz=r.Z-pZ
        #k=ln(I0/I)/dz
        mk=np.log(pPAR/r.PAR)/dz
        # append row to output dataframe here:
    # set previous values
    pOID=r.OID
    pZ=r.Z
    pSA=r.SA
    pCT=r.CT
    pc=ic
    pPAR=r.PAR
    pFl=r.Fluor

In [ ]:

k=ln(I0/I)/dz

In [ ]:
qry=session.query(StationTBL.ID.label('StationTBLID'),ObsTBL.ID.label('ObsTBLID'),
                  StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
                StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),StationTBL.Lat,StationTBL.Lon,
                ZD.label('Z'),PAR.label('PAR'),SA.label('SA'),CT.label('CT')).\
        select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
        join(CalcsTBL,CalcsTBL.ObsTBLID==ObsTBL.ID).filter(and_(StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                            StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121),
                                            StationTBL.Lat<51,
                                            StationTBL.Include==True,ObsTBL.Include==True,
                                            ObsTBL.PAR>=0,
                                            StationTBL.StartYear>=dstart.year,
                                            or_(StationTBL.StartYear>dstart.year,StationTBL.StartMonth>=dstart.month),
                                            StationTBL.StartYear<=dend.year,
                                            or_(StationTBL.StartYear<dend.year,StationTBL.StartMonth<=dend.month)))

In [ ]:
df1=pd.DataFrame(qry.all())

In [ ]:
df1['dtUTC']=[dt.datetime(int(y),int(m),int(d))+dt.timedelta(hours=h) \
              for y,m,d,h in zip(df1['Year'],df1['Month'],df1['Day'],df1['Hour'])]

In [ ]:
with nc.Dataset('/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc') as mesh:
    tmask=np.copy(mesh.variables['tmask'][0,:,:,:])
    navlat=np.copy(mesh.variables['nav_lat'][:,:])
    navlon=np.copy(mesh.variables['nav_lon'][:,:])

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(df1['Lon'],df1['Lat'],'k.')
ax.contour(navlon,navlat,tmask[0,:,:],(0.5,))

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
len(df1)

In [ ]:
# do matches with vvl considered
data=et.matchData(df1,filemap,fdict,dstart,dend,namfmt,PATH,flen,method='vvlBin')

In [ ]:
len(data)

In [ ]:
data

In [ ]:
data.keys()

In [ ]:
data2=data.loc[:,['StationTBLID', 'ObsTBLID','Z', 'PAR', 'SA', 'CT', 'dtUTC', 'i','j','k', 'mod_PAR', 'mod_e3t',
       'mod_vosaline', 'mod_votemper', 'mod_diatoms', 'mod_flagellates','mod_ciliates', 'mod_Fraser_tracer']]

In [ ]:
data2.to_sql(
    runID+'_matchTBL',
    engine,
    if_exists='replace',
    index=False,
    chunksize=500,
    dtype={
        'StationTBLID': Integer, 
        'ObsTBLID': Integer,
        'Z':Float, 
        'PAR':Float,  
        'SA':Float, 
        'CT':Float, 
        'dtUTC':DateTime, 
        'i':Integer,'j':Integer,'k':Integer, 
        'mod_PAR':Float, 
        'mod_e3t':Float,
       'mod_vosaline':Float, 'mod_votemper':Float, 'mod_diatoms':Float, 'mod_flagellates':Float,
        'mod_ciliates':Float, 'mod_Fraser_tracer':Float
    }
)

In [ ]:
session.commit()

In [ ]:
session.close()

In [ ]:
engine.dispose()

In [ ]:
engine = create_engine('sqlite:///' + basedir + dbname, echo = False)
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
# existing tables:
StationTBL=Base.classes.StationTBL
ObsTBL=Base.classes.ObsTBL
CalcsTBL=Base.classes.CalcsTBL
HC201812TBL=Base.classes.HC201812_matchTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)

In [ ]:
Base.classes.

In [ ]:
data

In [ ]:
data.keys()

In [ ]:
data.drop(data[data.k<0].index,inplace=True)

In [ ]:
np.shape(tmask)

In [ ]:
fe3t=nc.Dataset('/results/SalishSea/nowcast-green.201812/01jan15/SalishSea_1h_20150101_20150101_carp_T.nc')
e3t=fe3t.variables['e3t'][5,:,500,300][tmask[:,500,300]==1]
zl=np.zeros((len(e3t),2))
zl[1:,0]=np.cumsum(e3t[:-1])
zl[:,1]=np.cumsum(e3t)


In [ ]:
ik=[iii for iii,hhh in enumerate(zl) if hhh[1]>5.1][0] 

In [ ]:
ik

In [ ]:
zl

In [ ]:
fe3t.variables['e3t']

In [ ]:
plt.plot(data['mod_PAR'],data['PAR'],'k.',ms=1)

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
fig,ax=plt.subplots(1,1,figsize=(5,5))
xx=data.loc[ii,['mod_PAR']].values
yy=data.loc[ii,['PAR']].values
ax.hist2d(xx[:,0],yy[:,0],bins=100,cmin=0,cmax=50);

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii,['mod_PAR']].values
xx=data.loc[ii,['PAR']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Direct Comparison: HC201812')

In [ ]:
data.keys()

In [ ]:
stas=data.groupby(by=['Year','Month','Day','Lat','Lon'])

In [ ]:
stas

In [ ]:
with nc.Dataset('/results/SalishSea/nowcast-green.201812/01jan16/SalishSea_1h_20160101_20160101_ptrc_T.nc') as ftemp:
    deptht=ftemp.variables['deptht'][:]

In [ ]:
deptht[5]

In [ ]:
data['mod_Z']=[deptht[ii] for ii in data['k']]

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(data['Z'],data['mod_Z'],'k.',ms=1)
ax.set_xlabel('Obs')
ax.set_ylabel('Model')

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
ik=data.k<10
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii&ik,['mod_PAR']].values
xx=data.loc[ii&ik,['PAR']].values
yz=data.loc[ii&ik,['mod_Z']].values
xz=data.loc[ii&ik,['Z']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Direct Comparison Upper 10m: HC201812')

In [ ]:
plt.hist(yz-xz)
plt.title('hist: model z - obs z')

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.plot(yz-xz,np.log10(yy[:,0]+1)-np.log10(xx[:,0]+1),'k.',ms=1)
ax.set_xlabel('Model Z - Obs Z')
ax.set_ylabel('log10(model PAR +1)-log10(obs PAR +1)')

In [ ]:
ii=(data['mod_PAR']>=0)&(data['PAR']>=0)
ik=data.k<2
fig,ax=plt.subplots(1,1,figsize=(5,5))
yy=data.loc[ii&ik,['mod_PAR']].values
xx=data.loc[ii&ik,['PAR']].values
yz=data.loc[ii&ik,['mod_Z']].values
xz=data.loc[ii&ik,['Z']].values
_,_,_,m=ax.hist2d(np.log10(xx[:,0]+1),np.log10(yy[:,0]+1),bins=100,norm=mpl.colors.LogNorm(), cmap=mpl.cm.gist_rainbow_r);
fig.colorbar(m,ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(0,4)
ax.set_aspect(1)
ax.plot((0,4),(0,4),'k-')
ax.set_xlabel('obs')
ax.set_ylabel('model')
ax.set_title('PAR Direct Comparison Upper 2m: HC201812')

In [ ]:
data.keys()

In [ ]:
temp=data.loc[(data['PAR']>10)&(data['mod_PAR']==0)&(data['Z']<3),['dtUTC','Lat','Lon','Z','mod_Z','Hour','staFile','obsFile']]

In [ ]:
temp

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(16,1))
ax.plot(temp['dtUTC'],temp['Z'],'k.')

In [ ]:
plt.hist(temp['Hour'])

In [ ]:
len(temp)

In [ ]:
np.unique(temp['staFile'])

In [ ]:
np.unique(temp['obsFile'])

In [ ]:
plt.hist(np.log10(data['PAR']+1),200);
plt.ylim(0,10000)

In [ ]:
10**.5-1

In [ ]:
data.loc[data['obsFile']=='req20181116/EO UBC November 16, 2018 (2017 data)/2017-63-0017.ctd',['dtUTC','PAR']]

nighttime; probably moonlight